<a href="https://colab.research.google.com/github/ankitrajmehta/pdf-Question-answer/blob/main/pdfqa_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install pypdf

In [3]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain

In [4]:
def pdf_to_vec(pdf):
  reader = PyPDFLoader(pdf)
  pages = reader.load()
  chunker = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=100)
  chunks = chunker.split_documents(pages)
  embeddings = HuggingFaceEmbeddings()
  vector_store = Chroma.from_documents(chunks, embedding=embeddings)
  return vector_store

In [5]:
db = pdf_to_vec('LOCATION OF PDF')

In [6]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HUGGINGFACE API KEY' #enter huggingface api key. You can get it by creating an account on huggingface.co

In [7]:
llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5})#google/flan-t5-xxl
chat_history=[]
model = ConversationalRetrievalChain.from_llm(llm = llm, retriever= db.as_retriever())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
query = input("Enter question: ")
result = model({"question": query, "chat_history": chat_history})
result["answer"]

In [ ]:
chat_history = [(query, result["answer"])]

In [ ]:
chat_history

In [8]:
repeat = True
while repeat:
  query = input("Enter question: ")
  result = model({"question": query, "chat_history": chat_history})
  print(result["answer"])
  chat_history = [(query, result["answer"])]
  r = input("To ask more question enter 'y' else print any other key: ")
  if r.lower() != 'y':
    repeat = False

Enter question: Who won the war?
the victorious Allies
To ask more question enter 'y' else print any other keyy
Enter question: Who were they?
the victorious Allies
To ask more question enter 'y' else print any other keyy
Enter question: who were the allies?
United States, the United Kingdom, and France) and the Soviet Union
To ask more question enter 'y' else print any other keyn
